In [1]:
#import all the necessary packages.
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup

import math
import time
import re
import os
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

import gensim

from gensim.models import Word2Vec, KeyedVectors
# from sklearn.metrics.pairwise import cosine_similarity
# import nltk
# from nltk.corpus import stopwords

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/Train.csv')
data = pd.read_csv('Train.csv')
print(data.shape)
data.head()

(31499, 3)


,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [4]:
def remove_dup(x):
    words = x.split(",")
    words = list(set(words))
    words = ["'%s'"%i.strip("' ") for i in words]
    return ','.join(words)

In [5]:
# # removing duplicate distractors and convert distract column to list
data.distractor = data.distractor.apply(remove_dup)
data.distractor = [i.split(',') for i in data.distractor]
data.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,['in the dining - room from 7:30 a. m. to 9:15...
1,It can be inferred from the passage that,The local government can deal with the problem...,"['Currently', 'relevant departments of the Sta..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"['blame Tommy for his failing grades', 'blame ..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,['idioms are the most important part in a lang...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,['Cut the wound and suck the poison out .']


In [8]:
data.explode('distractor')

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,'in the dining - room from 7:30 a. m. to 9:15 ...
0,Meals can be served,in rooms at 9:00 p. m.,'in the dining - room at 6:00 p. m.'
0,Meals can be served,in rooms at 9:00 p. m.,'outside the room at 3:00 p. m.'
1,It can be inferred from the passage that,The local government can deal with the problem...,'Currently'
1,It can be inferred from the passage that,The local government can deal with the problem...,'relevant departments of the State Council sho...
...,...,...,...
31497,"According to Galbraith , people feel disconten...",materialism has run wild in modern society,"'""public spending has n't been cut down as exp..."
31497,"According to Galbraith , people feel disconten...",materialism has run wild in modern society,'they are in fear of another Great Depression'
31498,The boy grew up with the smell of the herbs be...,he often mixed herbs for his mother,'he liked go play with herbs .'
31498,The boy grew up with the smell of the herbs be...,he often mixed herbs for his mother,'he studied at a herbal medicine store'


In [17]:
data.explode('distractor').reset_index(inplace=True)

In [ ]:
#converting to lower case
data = data.apply(lambda x: x.astype(str).str.lower())
import re
# replacing _' with '
data = data.apply(lambda x: x.astype(str).str.replace(r" '", r"'"))
# replacing ( n't) with (n't)
data = data.apply(lambda x: x.astype(str).str.replace(r" n't", r"n't"))
# replacing ' s with 's
data = data.apply(lambda x: x.astype(str).str.replace(r" 's", r"'s"))
# removing duplicate repeatitive distractors
data.distractor = data.distractor.apply(remove_dup)
data = pd.concat([data, data.distractor.str.split(",", expand=True)], axis = 1)
# removing speacial charecters
data = data.apply(lambda x: x.astype(str).str.replace(r"[^A-Za-z0-9\s]+", r""))

In [ ]:
# Extracting repeated questions and marking them as common questions
common_que = data.question.value_counts()
common_que = list(common_que[common_que>3].index.values)
common_que = [i.strip() for i in common_que]
'the passage is mainly about' in common_que

In [ ]:
question_strings = data.question
questions = data.question.apply(lambda x: x.split(" "))
options_strings = data.drop(['question', 'distractor'], axis=1)
options = options_strings.apply(lambda x: x.astype(str).str.split(" "), axis=1)

In [ ]:
X_questions = questions.to_numpy()
X_options = options.to_numpy()

In [ ]:
%%time
w2v_ques_model = Word2Vec(X_questions, size=100, workers=4)
# w2v_ques_model.save('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/ques_word2vec.model')
w2v_ques_model.save('ques_word2vec.model')

In [ ]:
X_options = X_options.flatten()
for index, i in enumerate(X_options):
    X_options[index] = list(filter(lambda a: a != '', i))
X_options = np.unique(X_options)
w2v_opt_model = Word2Vec(X_options, size=100, workers=4)
w2v_opt_model.save('opt_word2vec.model')
# w2v_opt_model.save('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/opt_word2vec.model')

In [ ]:
def get_word_vec(sentence, text_type):
    if text_type == 'question':
        model = w2v_ques_model
        vocab = list(w2v_ques_model.wv.vocab)
    else:
        model = w2v_opt_model
        vocab = list(w2v_opt_model.wv.vocab)

    vec = []
    for i in sentence:
        if i in vocab:
            vec.append(model[i])
        else:
            vec.append(np.ones(shape=(100,)))
    if len(sentence)==0:
        vec.append(np.ones(shape=(100,)))
    return np.mean(vec, axis=0)

In [ ]:
%%time
X_ques_vec = np.array([get_word_vec(xi, 'question') for xi in X_questions])
X_opt_vec = np.array([get_word_vec(xi, 'option') for xi in X_options])

In [ ]:
X_opt_vec.shape

In [ ]:
def get_top_n_similiarities(sentance, n, text_type):
    if text_type == 'question':
        train_data = X_questions
        train_data_vec = X_ques_vec
    else:
        train_data = X_options
        train_data_vec = X_opt_vec
    distances = []
    vec2 = get_word_vec(sentance.split(" "), text_type)
    for index, i in enumerate(train_data):
        vec1 = train_data_vec[index]
        d = np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
        if len(distances) < n:
            distances.append((i,d))
        else:
            distances.sort(key=lambda x: x[1], reverse=True)
            if (distances[-1][1]<d):
                distances.pop()
                distances.append((i,d))
    result = ''
    for index, i in enumerate(distances):
        sen = ' '.join(i[0])
        result = result + "'"+sen+"'"
        if index!=len(distances)-1:
            result+=","
    return result


In [ ]:
import re
def preprocess_string(st):
    st = st.lower()
    st = re.sub(r" '", r"'", st)
    st = re.sub(r" n't", r"n't", st)
    st = re.sub(r" 's", r"'s", st)
    st = re.sub(r"[^A-Za-z0-9\s]+", r"", st)

    return st


In [ ]:
%%time
s = "which of the following is true according to the passage"
print(get_top_n_similiarities(preprocess_string(s), 10, 'option'))

## Generating output file

In [ ]:
# test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/Results.csv')
test_data = pd.read_csv('Results.csv')
test_data.head()

In [ ]:
preprocess_string("what 's the main idea of the text ")

In [ ]:
%%time
test_data.distractor = test_data.distractor.apply(str)
from tqdm import tqdm
for index, i in enumerate(test_data.iterrows()):
    if(index>8080):
        question = preprocess_string(test_data.at[index, 'question'])
        answer = preprocess_string(test_data.at[index, 'answer_text'])
        res = get_top_n_similiarities(answer, 3, 'option')
        test_data.at[index, 'distractor'] = res
        if(index%500 ==0):
            print(str(index) + "--" +res)
test_data.to_csv("Results2.csv", index=False)

In [ ]:
test_data.iloc[8085]

In [ ]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# data.to_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/t1.csv")